In [1]:
# Importar librerías

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [ ]:
url = pd.read_csv('urls.csv')
url = url['urls'].tolist()


# Realizamos la solicitud HTTP para obtener el contenido de la página
response = requests.get(url[1])

# Verificamos si la solicitud fue exitosa (código 200)
if response.status_code == 200:
    # Analizamos el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')


else:
    print(f"Error al obtener la página, código de estado: {response.status_code}")


In [ ]:
# Ahora, extraemos los precios o cualquier dato que te interese
data = []


# Suponiendo que el precio está en los <td> con una clase específica
# Este es solo un ejemplo, debes ajustar los selectores según el HTML que encuentres
for td in soup.find_all("td", class_="datatable_cell__LJp3C"):  # Cambia según la clase que encuentres
    precio = td.text.strip()
    data.append(precio)



In [24]:

# Crear una lista de tuplas de 3 elementos: fecha, precio, variación
chunks = [(data[i], data[i+1], data[i+2]) for i in range(0, len(data), 3)]

# Convertirlo a un DataFrame
df = pd.DataFrame(chunks, columns=['Fecha', 'Precio', 'Variación'])

# Convertir la columna 'Fecha' a tipo datetime
# Porque de verdad que es asqueroso el formato que usan los gringos
# ¿¡Qué es esa aberración de poner Mes - Día - Año!?
# O sea, va de mayor a menor a más mayor... que imbecilidad más grande
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%b %d, %Y')
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d')

# Convertir la columna 'Precio' a número flotante
df['Precio'] = pd.to_numeric(df['Precio'].str.replace(',', ''), errors='coerce')

# Convertir la columna 'Variación' a número flotante, eliminando el símbolo '%' y convirtiéndolo a tipo numérico
df['Variación'] = df['Variación'].str.replace('%', '').astype(float)

# Ordenar de fecha más antigua a más reciente.
df = df.sort_values(by='Fecha', ascending=True)
print (df)

        Fecha   Precio  Variación
19 2025-02-24  7311.33      -0.01
18 2025-02-25  7316.01       0.06
17 2025-02-26  7351.06       0.48
16 2025-02-27  7392.85       0.57
15 2025-02-28  7332.12      -0.82
14 2025-03-03  7344.96       0.18
13 2025-03-04  7313.89      -0.42
12 2025-03-05  7333.10       0.26
11 2025-03-06  7370.32       0.51
10 2025-03-07  7377.09       0.09
9  2025-03-10  7369.67      -0.10
8  2025-03-11  7365.82      -0.05
7  2025-03-12  7391.30       0.35
6  2025-03-13  7449.87       0.79
5  2025-03-14  7509.00       0.79
4  2025-03-17  7601.82       1.24
3  2025-03-18  7585.61      -0.21
2  2025-03-19  7594.95       0.12
1  2025-03-20  7575.53      -0.26
0  2025-03-21  7585.81       0.14


In [34]:
ipsaCompleto = pd.read_csv('equities/ipsaCompleto.csv')

ipsaCompleto['Fecha'] = pd.to_datetime(ipsaCompleto['Fecha'], format='%Y-%m-%d')

# Verificar cómo quedaron las fechas
print(ipsaCompleto.head())





       Fecha  Valor IPSA
0 1993-09-27      538.01
1 1993-09-28      541.05
2 1993-09-29      541.05
3 1993-09-30      539.48
4 1993-10-01      541.61


In [38]:
# Asegurarnos de que las fechas de los nuevos datos estén también en formato de fecha
#df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d')


# Iterar sobre las fechas de los nuevos datos y buscar si existen en los históricos
for _, row in df.iterrows():
    # Buscar si la fecha del nuevo DataFrame existe en el histórico
    historical_row = ipsaCompleto.loc[ipsaCompleto['Fecha'] == row['Fecha']]

    if not historical_row.empty:
        # Si encontramos la fecha, comparar los precios
        historical_price = historical_row['Valor IPSA'].values[0]
        if historical_price != row['Precio']:
            print(f"¡Precio diferente en {row['Fecha']}! Nuevo: {row['Valor IPSA']} - Histórico: {historical_price}")
        else:
            print (f"Los precios en {row['Fecha']} son {row['Precio']}, y están bien con el histórico, que es {historical_price} :D")
    else:
        print(f"¡Fecha {row['Fecha']} no encontrada en los datos históricos!")

Los precios en 2025-02-24 00:00:00 son 7311.33, y están bien con el histórico, que es 7311.33 :D
Los precios en 2025-02-25 00:00:00 son 7316.01, y están bien con el histórico, que es 7316.01 :D
Los precios en 2025-02-26 00:00:00 son 7351.06, y están bien con el histórico, que es 7351.06 :D
Los precios en 2025-02-27 00:00:00 son 7392.85, y están bien con el histórico, que es 7392.85 :D
Los precios en 2025-02-28 00:00:00 son 7332.12, y están bien con el histórico, que es 7332.12 :D
Los precios en 2025-03-03 00:00:00 son 7344.96, y están bien con el histórico, que es 7344.96 :D
Los precios en 2025-03-04 00:00:00 son 7313.89, y están bien con el histórico, que es 7313.89 :D
Los precios en 2025-03-05 00:00:00 son 7333.1, y están bien con el histórico, que es 7333.1 :D
Los precios en 2025-03-06 00:00:00 son 7370.32, y están bien con el histórico, que es 7370.32 :D
Los precios en 2025-03-07 00:00:00 son 7377.09, y están bien con el histórico, que es 7377.09 :D
Los precios en 2025-03-10 00:00: